# Lesson 2.6:
# Simulation API

This tutorial introduces the Simulation API and how it can be used to run power system simulations, subscribe to measurement data, and publish equipment control commands.

__Learning Objectives:__

At the end of the tutorial, the user should be able to use the Simulation API to

* 
* 
* 

## Getting Started

Before running any of the sample routines in this tutorial, it is first necessary to start the GridAPPS-D Platform and establish a connection to this notebook so that we can start passing calls to the API.

_Open the Ubuntu terminal and start the GridAPPS-D Platform if it is not running already:_

`cd gridappsd-docker`

~/gridappsd-docker$ `./run.sh -t develop`

_Once containers are running,_

gridappsd@[container]:/gridappsd$ `./run-gridappsd.sh`

In [1]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
gapps = GridAPPSD("('localhost', 61613)", username='system', password='manager')
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62" # IEEE 13 Node used for all example queries

---
# Table of Contents

* [1. Introduction to the PowerGrid Model API](#1.-Introduction-to-the-PowerGrid-Model-API)
* [2. Using the PowerGrid Model API](#2.-Using-the-PowerGrid-Model-API)
    * [2.1. Specifying the Topic](#2.1.-Specifying-the-Topic)
        * [2.1.1. Topic to Start a Simulation](#2.1.1.-Topic-to-Start-a-Simulation)
        * [2.1.2. Topic to Pause, Resume, or Stop a Simulation](#2.1.2.-Topic-to-Pause,-Resume,-or-Stop-a-Simulation)
        * [2.1.3. Topic to Subscribe to Simulation Output](#2.1.3.-Topic-to-Subscribe-to-Simulation-Output)
        * [2.1.4. Topic to Publish to Simulation Input](#2.1.4.-Topic-to-Publish-to-Simulation-Input)
        * [2.1.5. Topic to Subscribe to Simulation Logs](#2.1.5.-Topic-to-Subscribe-to-Simulation-Logs)
    * [2.2. Structure of a JSON Query Message](#2.2.-Structure-of-a-JSON-Query-Message)
    * [2.3. Specifying the requestType](#2.3.-Specifying-the-requestType)
* [3. Querying for Feeder Model Info](#3.-Querying-for-Feeder-Model-Info)
    * [3.1. Query for mRIDs of all Models](#3.1.-Query-for-mRIDs-of-all-Models)
    * [3.2. Query for Details Dictionary of all Models](#3.2.-Query-for-Details-Dictionary-of-all-Models)
* [4. Querying for Object Info](#4.-Querying-for-Object-Info)
    * [4.1. Query for CIM Classes of Objects in Model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)

---
# 1. Introduction to the Simulation API


The Simulation API is used for all actions related to a power system simulation. It is used to start, pause, restart, and stop a simulation from the command line or inside an application. It is all used to subscribe to measurements of equipment (such as line flows, loads, and DG setpoints) and the statuses of switches, capacitors, transformer taps, etc. It is also used to publish equipment control and other simulation input commands.

---
# 2. Using the Simulation API

## 2.1. Specifying the Topic

In the previous lessons all the queries passed to other APIs all used static `/queue/` channels to pass API calls to the GOSS message bus and the GridAPPS-D Platform. 

However, the Simulation API is used to communicate with a broad range of subscribers. The list of topics covered by the Simulation API cover both static `/queue/` and dynamic `/topic/` communication channel names. 

Extreme care is needed to use the correct topic. For a review of GridAPPS-D topics, see Lesson 1.4.

### 2.1.1. Topic to Start a Simulation

This is a static `/queue/` communication channel

1) Specifying the topic as a string:

* `topic = "goss.gridappsd.process.request.simulation"`

2) Using the topics library to specify the topic:

* `from gridappsd import topics as t`
* `topic = t.REQUEST_SIMULATION`

### 2.1.2. Topic to Pause, Resume, or Stop a Simulation

This is a dynamic `/topic/` communication channel that is best implemented by importing the GriAPPSD-Python library function for generating the correct topic. 

* `from gridappsd import simulation_input_topic`
* `topic = simulation_input_topic(simulation_id)`

### 2.1.3. Topic to Subscribe to Simulation Output

This is a dynamic `/topic/` communication channel that is best implemented by importing the GriAPPSD-Python library function for generating the correct topic. 

* `from gridappsd import simulation_output_topic`
* `topic = simulation_output_topic(simulation_id)`

### 2.1.4. Topic to Publish to Simulation Input

This is a dynamic `/topic/` communication channel that is best implemented by importing the GriAPPSD-Python library function for generating the correct topic. 

* `from gridappsd import simulation_input_topic`
* `topic = simulation_input_topic(simulation_id)`

### 2.1.5. Topic to Subscribe to Simulation Logs

This is a dynamic `/topic/` communication channel that is best implemented by importing the GriAPPSD-Python library function for generating the correct topic. 

* `from gridappsd import simulation_log_topic`
* `topic = simulation_log_topic(simulation_id)`

[Return to Top](#Table-of-Contents)

## 2.2. Structure of a JSON Query Message


Queries passed to Timeseries API are formatted as python dictionaries or equivalent JSON scripts wrapped as a python string. 

The accepted set of key-value pairs for the 

```
message = """
{
    "requestType": "INSERT QUERY HERE",
    "resultFormat": "JSON",
    "modelId": "OPTIONAL INSERT MODEL mRID HERE",
    "objectId": "OPTIONAL INSERT OBJECT mRID HERE",
    "filter": "OPTIONAL INSERT SPARQL FILTER HERE"
}
```

The components of the message are as follows:

* `"requestType":` -- Specifies the type of query. Available requestType are listed in the next section.


* `"resultFormat":` -- Specifies the format of the response, can be `"JSON"`, `"CSV"`, or `"XML"`. (CAUTION: the PowerGridModel API uses the key _resultFormat_, while the Timeseries API uses the key _reponseFormat_. Using the wrong key for either API will result in a java.lang error.)


* `"modelID":` -- Optional. Used to filter the query to only one particular model whose mRID is specified. Be aware of spelling and capitalization differences between JSON query spelling `"modelId"` and Python Library spelling `model_id`.


* `"objectType":` -- Optional. Used to filter the query to only one CIM class of equipment. Speciying the _objectID_ will override any values specified for _objectType_. 


* `"objectID":` -- Optional. Used to filter the query to only one object whose mRID is specified. Specifying the _objectID_ will override any values specified for _objectType_. 


* `"filter":` -- Optional. Used to filter the query using a SPARQL filter. SPARQL queries are covered in the next lesson.


The usage of each of these message components are explained in detail with code block examples below. 

__Important__: Be sure to pay attention to placement of commas ( __,__ ) at the end of each JSON line. Commas are placed at the end of each line _except_ the last line. Incorrect comma placement will result in a JsonSyntaxException. 

All of the queries are passed to the PowerGrid Model API using the `.get_response(topic, message)` method for the GridAPPS-D platform connection variable.

[Return to Top](#Table-of-Contents)

## 2.3. Specifying the `requestType`

Below are the possible `requestType` strings that are used to specify the type of each query. Executable code block examples are provided for each of the requests in the subsections below.

The first group of _requestType_ are for queries for information related to the entire model or a set of models, such as the model name, mRID, region, and substation:

* `"requestType": "QUERY_MODEL_NAMES"` -- [Query for the list of all model name mRIDs](#3.1.-Query-for-mRIDs-of-all-Models)


* `"requestType": "QUERY_MODEL_INFO"` -- [Query for the dictionary of all details for all feeders in Blazegraph](#3.2.-Query-for-Details-Dictionary-of-all-Models)


The second group of _requestType_ are for queries for a single object or a single class of objects withing a model, such as the object mRID, CIM attributes, or measurement points:

* `"requestType": "QUERY_OBJECT_TYPES"` -- [Query for the types of CIM classes of objects in the model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)


* `"requestType": "QUERY_OBJECT_IDS"` -- Query for a list of all mRIDs for objects of a CIM class in the model


* `"requestType": "QUERY_OBJECT"` -- Query for CIM attributes of an object using its unique mRID 


* `"requestType": "QUERY_OBJECT_DICT"` -- Query for the dictionary of all details for an object using either its _objectType_ OR its _objectID_


* `"requestType": "QUERY_OBJECT_MEASUREMENTS"` -- Query for all measurement types and mRIDs for an object using either its _objectType_ OR its _ObjectID_.

The third group of _requestType_ are for queries based on SPARQL filters or complete SPARQL queries. The structure of SPARQL was introduced in [Lesson 1.XX](). Usage of these two _requestType_ will covered separately in the next two lessons.

* `"requestType": "QUERY_MODEL"` -- Query for all  part of a specified model, filtered by object type using a SPARQL filter.


* `"requestType": "QUERY"` -- Query using a complete SPARQL query.




[[Return to Top](#Table-of-Contents)]

In [ ]:
%run Run123NodeDemo.py

In [ ]:
from gridappsd import utils

sim_id = utils.get_gridappsd_simulation_id()

print(sim_id)

In [1]:
import Run123NodeDemo as r
simulation_id = r.simulation.simulation_id

In [2]:
simulation_id

'1101066887'